In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import h5py

import torch
from torch.utils.data import Dataset

In [2]:
from dataset import ProcessedLigandPocketDataset
from pathlib import Path
import torch_geometric.transforms as T

datadir = '../data/docking_results/processed_crossdock_noH_full_temp'
data_transform = None

train_dataset = ProcessedLigandPocketDataset(Path(datadir, 'train.npz'), transform=data_transform)
test_dataset = ProcessedLigandPocketDataset(Path(datadir, 'test.npz'), transform=data_transform)
val_dataset = ProcessedLigandPocketDataset(Path(datadir, 'val.npz'), transform=data_transform)


In [3]:
train_dataset[0].keys()

dict_keys(['names', 'prompt_labels', 'lig_coords', 'lig_one_hot', 'lig_bonds', 'lig_mask', 'num_lig_atoms', 'pocket_coords', 'pocket_one_hot', 'pocket_mask', 'num_pocket_nodes'])

In [4]:
train_dataset[1]['prompt_labels'].size()

torch.Size([26, 3])

In [5]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=8, num_workers=24, collate_fn=train_dataset.collate_fn, shuffle=False, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=8, num_workers=24, collate_fn=val_dataset.collate_fn, shuffle=False,pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=8, num_workers=24, collate_fn=test_dataset.collate_fn, shuffle=False,pin_memory=True)

In [6]:
for batch in train_loader:
    print(batch.keys())
    break

dict_keys(['names', 'prompt_labels', 'lig_coords', 'lig_one_hot', 'lig_bonds', 'lig_mask', 'num_lig_atoms', 'pocket_coords', 'pocket_one_hot', 'pocket_mask', 'num_pocket_nodes'])


In [7]:
for i, batch in enumerate(train_loader):
    if i == 1:  # 0-based index, so this is the second batch
        print(batch['names'])
        break

['1bpy_A_rec_4uaz_8dg_lig_tt_min/1bpy_A_rec_4uaz_8dg_lig_tt_min_0_pocket10.pdb_1bpy_A_rec_4uaz_8dg_lig_tt_min/1bpy_A_rec_4uaz_8dg_lig_tt_min_generated_1_docked_poses_minimized.sdf', '4y95_A_rec_5p9i_1e8_lig_tt_docked/4y95_A_rec_5p9i_1e8_lig_tt_docked_1_pocket10.pdb_4y95_A_rec_5p9i_1e8_lig_tt_docked/4y95_A_rec_5p9i_1e8_lig_tt_docked_generated_5_docked_poses_minimized.sdf', '1pdb_A_rec_5hsr_63y_lig_tt_docked/1pdb_A_rec_5hsr_63y_lig_tt_docked_13_pocket10.pdb_1pdb_A_rec_5hsr_63y_lig_tt_docked/1pdb_A_rec_5hsr_63y_lig_tt_docked_5.sdf', '2jcs_A_rec_2jcs_ttp_lig_tt_min/2jcs_A_rec_2jcs_ttp_lig_tt_min_0_pocket10.pdb_2jcs_A_rec_2jcs_ttp_lig_tt_min/2jcs_A_rec_2jcs_ttp_lig_tt_min_0.sdf', '3nhp_A_rec_4qoh_stl_lig_tt_min/3nhp_A_rec_4qoh_stl_lig_tt_min_0_pocket10.pdb_3nhp_A_rec_4qoh_stl_lig_tt_min/3nhp_A_rec_4qoh_stl_lig_tt_min_generated_5_docked_poses_minimized.sdf', '3olg_A_rec_1xcw_3sa_lig_tt_docked/3olg_A_rec_1xcw_3sa_lig_tt_docked_0_pocket10.pdb_3olg_A_rec_1xcw_3sa_lig_tt_docked/3olg_A_rec_1xcw_3

In [8]:
from constants import dataset_params, FLOAT_TYPE, INT_TYPE
def get_ligand_and_pocket(data,virtual_nodes):
    ligand = {
        'x': data['lig_coords'].to('cuda', FLOAT_TYPE),
        'one_hot': data['lig_one_hot'].to('cuda', FLOAT_TYPE),
        'size': data['num_lig_atoms'].to('cuda', INT_TYPE),
        'mask': data['lig_mask'].to('cuda', INT_TYPE),
    }
    if virtual_nodes:
        ligand['num_virtual_atoms'] = data['num_virtual_atoms'].to('cuda', INT_TYPE)
    
    pocket = {
        'x': data['pocket_coords'].to('cuda', FLOAT_TYPE),
        'one_hot': data['pocket_one_hot'].to('cuda', FLOAT_TYPE),
        'size': data['num_pocket_nodes'].to('cuda', INT_TYPE),
        'mask': data['pocket_mask'].to('cuda', INT_TYPE)
    }
    return ligand, pocket

In [9]:
ligand, pocket = get_ligand_and_pocket(batch, virtual_nodes=False)

In [10]:
ligand['x']

tensor([[ 2.6650e+00,  6.8896e-01, -6.3784e-01],
        [ 1.6390e+00,  1.5570e+00, -1.3338e+00],
        [ 2.4870e+00,  2.8170e+00, -1.5678e+00],
        [ 3.3280e+00,  2.5450e+00, -2.6528e+00],
        [ 3.2780e+00,  2.8270e+00, -2.7584e-01],
        [ 3.6150e+00,  1.5130e+00, -9.7843e-02],
        [ 4.6710e+00, -4.4200e+00,  1.5157e-02],
        [ 3.6290e+00, -3.6280e+00,  5.9116e-01],
        [ 2.8790e+00, -4.0980e+00,  1.5172e+00],
        [ 1.8300e+00, -3.3690e+00,  2.1292e+00],
        [ 1.0940e+00, -3.8270e+00,  3.0362e+00],
        [ 1.6160e+00, -2.0000e+00,  1.6552e+00],
        [ 7.7103e-01, -1.0830e+00,  1.9862e+00],
        [ 1.0060e+00,  7.7959e-02,  1.1972e+00],
        [ 3.6103e-01,  1.1560e+00,  1.2652e+00],
        [ 2.0830e+00, -1.8404e-01,  3.2916e-01],
        [ 2.4980e+00, -1.5170e+00,  6.0016e-01],
        [ 3.3980e+00, -2.2970e+00,  1.5016e-01],
        [ 4.9820e+00,  4.0950e+00, -4.1484e-01],
        [ 4.8310e+00,  5.7160e+00,  7.6616e-01],
        [ 4.3350e+00

In [11]:

xh_lig = torch.cat([ligand['x'], ligand['one_hot']], dim=1)
xh_pocket = torch.cat([pocket['x'], pocket['one_hot']], dim=1)
xh_pocket.size()

torch.Size([3114, 14])

In [12]:
ligand['size']

tensor([32, 33, 18, 29, 18, 33, 28, 21], device='cuda:0')

In [13]:
def sigma(gamma, target_tensor):
        """Computes sigma given gamma."""
        return inflate_batch_array(torch.sqrt(torch.sigmoid(gamma)),
                                        target_tensor)
    
def inflate_batch_array(array, target):
    """
    Inflates the batch array (array) with only a single axis
    (i.e. shape = (batch_size,), or possibly more empty axes
    (i.e. shape (batch_size, 1, ..., 1)) to match the target shape.
    """
    target_shape = (array.size(0),) + (1,) * (len(target.size()) - 1)
    return array.view(target_shape)
class PositiveLinear(torch.nn.Module):
    """Linear layer with weights forced to be positive."""

    def __init__(self, in_features: int, out_features: int, bias: bool = True,
                 weight_init_offset: int = -2):
        super(PositiveLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = torch.nn.Parameter(
            torch.empty((out_features, in_features)))
        if bias:
            self.bias = torch.nn.Parameter(torch.empty(out_features))
        else:
            self.register_parameter('bias', None)
        self.weight_init_offset = weight_init_offset
        self.reset_parameters()

    def reset_parameters(self) -> None:
        torch.nn.init.kaiming_uniform_(self.weight, a=math.sqrt(5))

        with torch.no_grad():
            self.weight.add_(self.weight_init_offset)

        if self.bias is not None:
            fan_in, _ = torch.nn.init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
            torch.nn.init.uniform_(self.bias, -bound, bound)

    def forward(self, input):
        positive_weight = F.softplus(self.weight)
        return F.linear(input, positive_weight, self.bias)

class GammaNetwork(torch.nn.Module):
    """The gamma network models a monotonic increasing function.
    Construction as in the VDM paper."""
    def __init__(self):
        super().__init__()

        self.l1 = PositiveLinear(1, 1)
        self.l2 = PositiveLinear(1, 1024)
        self.l3 = PositiveLinear(1024, 1)

        self.gamma_0 = torch.nn.Parameter(torch.tensor([-5.]))
        self.gamma_1 = torch.nn.Parameter(torch.tensor([10.]))
        self.show_schedule()

    def show_schedule(self, num_steps=50):
        t = torch.linspace(0, 1, num_steps).view(num_steps, 1)
        gamma = self.forward(t)
        print('Gamma schedule:')
        print(gamma.detach().cpu().numpy().reshape(num_steps))

    def gamma_tilde(self, t):
        l1_t = self.l1(t)
        return l1_t + self.l3(torch.sigmoid(self.l2(l1_t)))

    def forward(self, t):
        zeros, ones = torch.zeros_like(t), torch.ones_like(t)
        # Not super efficient.
        gamma_tilde_0 = self.gamma_tilde(zeros)
        gamma_tilde_1 = self.gamma_tilde(ones)
        gamma_tilde_t = self.gamma_tilde(t)

        # Normalize to [0, 1]
        normalized_gamma = (gamma_tilde_t - gamma_tilde_0) / (
                gamma_tilde_1 - gamma_tilde_0)

        # Rescale to [gamma_0, gamma_1]
        gamma = self.gamma_0 + (self.gamma_1 - self.gamma_0) * normalized_gamma

        return gamma
   
import math
import torch.nn.functional as F
gamma = GammaNetwork()
gamma.to('cuda')
lowest_t = 0
time_step = 1000
t_int = torch.randint(
    lowest_t, time_step + 1, size=(ligand['size'].size(0), 1),
    device=ligand['x'].device).float()
s_int = t_int - 1  # previous timestep
# Masks: important to compute log p(x | z0).
t_is_zero = (t_int == 0).float()
t_is_not_zero = 1 - t_is_zero
# Normalize t to [0, 1]. Note that the negative
# step of s will never be used, since then p(x | z0) is computed.
s = s_int / time_step
t = t_int / time_step
# Compute gamma_s and gamma_t via the network.
gamma_s = inflate_batch_array(gamma(s), ligand['x'])
gamma_t = inflate_batch_array(gamma(t), ligand['x'])
sigma_s = sigma(gamma_s, xh_lig)
lig_mask = ligand["mask"]
sigma_s[lig_mask].size()


Gamma schedule:
[-5.         -4.6935     -4.3872995  -4.081174   -3.774674   -3.4683237
 -3.1620486  -2.8557732  -2.5495727  -2.2430727  -1.9369473  -1.6305969
 -1.3243966  -1.0181212  -0.7118459  -0.4057207  -0.09952021  0.20675516
  0.51295567  0.81915617  1.1252818   1.431407    1.7376075   2.0438828
  2.3497834   2.6561341   2.9621096   3.2682343   3.5743608   3.8804111
  4.186387    4.4926624   4.7986383   5.1046886   5.4108133   5.716714
  6.022765    6.3286667   6.634716    6.9406176   7.246668    7.5525694
  7.8586187   8.16452     8.470421    8.776321    9.082373    9.3881235
  9.694168   10.        ]


torch.Size([212, 1])

In [14]:
pocket['mask'].size()

torch.Size([3114])

In [15]:
xh_lig = torch.cat([ligand['x'], ligand['one_hot']], dim=1)
xh_pocket = torch.cat([pocket['x'], pocket['one_hot']], dim=1)
zs_lig, zs_pocket = xh_lig, xh_pocket
shift_net_out_ligand = torch.ones(57,14)
shift_net_out_ligand = shift_net_out_ligand.to(zs_lig.device)
shift_net_out_pocket = torch.ones(740,14)
shift_net_out_pocket = shift_net_out_pocket.to(zs_pocket.device)
print(zs_lig.device, shift_net_out_ligand.device)
ligand_mask = ligand["mask"]
pocket_mask = pocket["mask"]
zt_lig = zs_lig + shift_net_out_ligand[ligand_mask]*zs_lig*(1/time_step) + sigma_s[ligand_mask]*(1.0/time_step)**(1/2)*torch.randn_like(zs_lig)


cuda:0 cuda:0


In [16]:
print(zt_lig.size())
print(zs_lig.size())

torch.Size([212, 14])
torch.Size([212, 14])


In [17]:
(ligand_mask).size()

torch.Size([212])

In [18]:
def sigma(gamma, target_tensor):
        """Computes sigma given gamma."""
        return inflate_batch_array(torch.sqrt(torch.sigmoid(gamma)),
                                        target_tensor)

def alpha(gamma, target_tensor):
    """Computes alpha given gamma."""
    return inflate_batch_array(torch.sqrt(torch.sigmoid(-gamma)),
                                    target_tensor)
def sample_gaussian(size, device):
        x = torch.randn(size, device=device)
        return x

In [19]:
alpha_t = alpha(gamma_t, xh_lig)
sigma_t = sigma(gamma_t, xh_lig)

# Sample zt ~ Normal(alpha_t x, sigma_t)
eps_lig = sample_gaussian(
    size=(len(lig_mask), 3 + 11),
    device=lig_mask.device)

# Sample z_t given x, h for timestep t, from q(z_t | x, h)
z_t_lig = alpha_t[lig_mask] * xh_lig + sigma_t[lig_mask] * eps_lig

In [20]:

from constants import dataset_params
import utils
from utils import AppendVirtualNodes

dataset_info = dataset_params['crossdock_full']
histogram_file = Path(datadir, 'size_distribution.npy')
histogram = np.load(histogram_file).tolist()



lig_type_encoder = dataset_info['atom_encoder']
lig_type_decoder = dataset_info['atom_decoder']
pocket_type_encoder = dataset_info['aa_encoder']
pocket_type_decoder = dataset_info['aa_decoder']

virtual_nodes = True
data_transform = None
max_num_nodes = len(histogram) - 1
if virtual_nodes:
    # symbol = 'virtual'

    symbol = 'Ne'  # visualize as Neon atoms
    lig_type_encoder[symbol] = len(lig_type_encoder)
    data_transform = utils.AppendVirtualNodes(
        max_num_nodes, lig_type_encoder, symbol)
    
    virtual_atom = lig_type_encoder[symbol]
    lig_type_decoder.append(symbol)


    # Update dataset_info dictionary. This is necessary for using the
    # visualization functions.
    dataset_info['atom_encoder'] = lig_type_encoder
    dataset_info['atom_decoder'] = lig_type_decoder

atom_nf = len(lig_type_decoder)
aa_nf = len(pocket_type_decoder)

In [21]:
atom_nf

12

In [22]:
histogram

[[0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,

In [23]:
max_num_nodes

46

In [24]:
lig_type_decoder

['C', 'N', 'O', 'S', 'B', 'Br', 'Cl', 'P', 'I', 'F', 'others', 'Ne']

In [25]:
train_dataset = ProcessedLigandPocketDataset(Path(datadir, 'train.npz'), transform=data_transform)
test_dataset = ProcessedLigandPocketDataset(Path(datadir, 'test.npz'), transform=data_transform)
val_dataset = ProcessedLigandPocketDataset(Path(datadir, 'val.npz'), transform=data_transform)


train_loader = DataLoader(train_dataset, batch_size=8, num_workers=24, collate_fn=train_dataset.collate_fn, shuffle=False, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=8, num_workers=24, collate_fn=val_dataset.collate_fn, shuffle=False,pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=8, num_workers=24, collate_fn=test_dataset.collate_fn, shuffle=False,pin_memory=True)

In [26]:
from equivariant_diffusion.dynamics import EGNNDynamics

x_dims = 3
joint_nf =4

condition_vector = torch.tensor([0, 0, 1], dtype=torch.int, device='cuda')
condition_vector = condition_vector.unsqueeze(0).expand(2, -1)


net_dynamics = EGNNDynamics(
    atom_nf = atom_nf,
    residue_nf = aa_nf,
    n_dims = x_dims,
    joint_nf = joint_nf,
    device='cuda',
    hidden_nf=2,
    act_fn=torch.nn.SiLU(),
    n_layers= 2 ,
    attention= True,
    tanh=True,
    norm_constant=1,
    inv_sublayers=1,
    sin_embedding=False,
    normalization_factor=100,
    aggregation_method= 'sum' ,
    edge_cutoff_ligand=10,
    edge_cutoff_pocket=4,
    edge_cutoff_interaction=4,
    update_pocket_coords= False,
    reflection_equivariant=True,
    edge_embedding_dim=8,
    condition_vector = True
)


In [27]:
histogram_file = Path(datadir, 'size_distribution.npy')
histogram_file


PosixPath('../data/docking_results/processed_crossdock_noH_full_temp/size_distribution.npy')

In [28]:
from equivariant_diffusion.conditional_model import ConditionalDDPM


x_dims = 3
joint_nf =4


cddpm = ConditionalDDPM(
            dynamics = net_dynamics,
            atom_nf = atom_nf,
            residue_nf = aa_nf,
            n_dims = x_dims,
            timesteps= 100,
            noise_schedule = 'polynomial_2',
            noise_precision = 5.0e-4,
            loss_type = 'l2',
            norm_values = [1, 4],
            size_histogram = histogram,
            virtual_node_idx=lig_type_encoder[symbol] if virtual_nodes else None
    )




Entropy of n_nodes: H[N] 8.51980972290039


In [29]:
dynamics = net_dynamics,
atom_nf = atom_nf,
residue_nf = aa_nf,
n_dims = x_dims,
timesteps= 100,
noise_schedule = 'polynomial_2',
noise_precision = 5.0e-4,
loss_type = 'l2',
norm_values = [1, 4],
size_histogram = histogram,
virtual_node_idx=lig_type_encoder[symbol] if virtual_nodes else None

In [32]:
def get_prompts(data):
    # 创建一个张量 [0, 0, 1]
    prompts = torch.tensor(data['prompt_labels']).to('cuda', INT_TYPE)
    
    return prompts

In [34]:
loss_type = 'l2'

In [35]:
import torch
from tqdm import tqdm

# 假设你已经定义了模型、优化器和其他超参数
optimizer = torch.optim.Adam(cddpm.parameters(), lr=0.001)  # 选择合适的学习率
num_epochs = 1
device = 'cuda'

for epoch in range(num_epochs):
    cddpm.train()  # 设置模型为训练模式
    cddpm.to(device)
    total_loss = 0
    pbar = tqdm(train_loader, desc=f'Epoch {epoch}', leave=False)

    for batch in pbar:
        # 假设 batch 是一个列表，包含多个样本
        # 将每个样本移动到设备并组合成字典
        # batch={key:batch[key].cuda() for key in batch}

        
        optimizer.zero_grad()  # 清空梯度

        ligand, pocket = get_ligand_and_pocket(batch,virtual_nodes)
        prompt_labels = get_prompts(batch)

        
        loss_terms = cddpm(ligand, pocket, prompt_labels, return_info=False)

        delta_log_px, error_t_lig, error_t_pocket, SNR_weight, \
        loss_0_x_ligand, loss_0_x_pocket, loss_0_h, neg_log_const_0, \
        kl_prior, log_pN, t_int, xh_lig_hat, info = \
            cddpm(ligand, pocket,prompt_labels, return_info=True)
        if loss_type == 'l2':
            actual_ligand_size = ligand['size'] - ligand['num_virtual_atoms'] if virtual_nodes else ligand['size']

            # normalize loss_t
            denom_lig = x_dims * actual_ligand_size + \
                        cddpm.atom_nf * ligand['size']
            error_t_lig = error_t_lig / denom_lig
            denom_pocket = (x_dims + cddpm.residue_nf) * pocket['size']
            error_t_pocket = error_t_pocket / denom_pocket
            loss_t = 0.5 * (error_t_lig + error_t_pocket)


            # normalize loss_0
            loss_0_x_ligand = loss_0_x_ligand / (x_dims * actual_ligand_size)
            loss_0_x_pocket = loss_0_x_pocket / (x_dims * pocket['size'])
            loss_0 = loss_0_x_ligand + loss_0_x_pocket + loss_0_h


        nll = loss_t + loss_0 + kl_prior

        # print("loss", nll)
        nll = nll.mean()

        # 反向传播
        nll.backward()

        # 更新参数
        optimizer.step()

        # 累加损失
        total_loss += nll.item()
        pbar.set_postfix(nll_loss=nll.item())  # 更新进度条的后缀信息

    print(f'Epoch {epoch}, Average NLL Loss: {total_loss / len(train_loader)}')

Epoch 0:   0%|          | 0/58 [00:00<?, ?it/s]/tmp/ipykernel_655278/3916911146.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  prompts = torch.tensor(data['prompt_labels']).to('cuda', INT_TYPE)
                                                                        

Epoch 0, Average NLL Loss: 0.7734084273206776


In [38]:
import torch
from tqdm import tqdm
import os

# 假设你已经定义了模型、优化器和其他超参数
optimizer = torch.optim.Adam(cddpm.parameters(), lr=0.001)  # 选择合适的学习率
num_epochs = 1
device = 'cuda'
save_dir = '../checkpoints/cddpm'  # 模型保存的文件夹路径

# 创建保存目录（如果不存在）
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# 训练循环
for epoch in range(num_epochs):
    # 设置模型为训练模式
    cddpm.train()
    cddpm.to(device)
    total_loss = 0
    pbar = tqdm(train_loader, desc=f'Epoch {epoch}', leave=False)

    # 训练阶段
    for batch in pbar:
        # batch = {key: batch[key].to(device) for key in batch}

        optimizer.zero_grad()  # 清空梯度

        # 提取配体和口袋数据
        ligand, pocket = get_ligand_and_pocket(batch, virtual_nodes)
        
        # 计算损失，返回 (nll, info)
        delta_log_px, error_t_lig, error_t_pocket, SNR_weight, \
        loss_0_x_ligand, loss_0_x_pocket, loss_0_h, neg_log_const_0, \
        kl_prior, log_pN, t_int, xh_lig_hat, info = cddpm(ligand, pocket, return_info=True)

        if loss_type == 'l2':
            actual_ligand_size = ligand['size'] - ligand['num_virtual_atoms'] if virtual_nodes else ligand['size']

            # normalize loss_t
            denom_lig = x_dims * actual_ligand_size + \
                        cddpm.atom_nf * ligand['size']
            error_t_lig = error_t_lig / denom_lig
            denom_pocket = (x_dims + cddpm.residue_nf) * pocket['size']
            error_t_pocket = error_t_pocket / denom_pocket
            loss_t = 0.5 * (error_t_lig + error_t_pocket)

            # normalize loss_0
            loss_0_x_ligand = loss_0_x_ligand / (x_dims * actual_ligand_size)
            loss_0_x_pocket = loss_0_x_pocket / (x_dims * pocket['size'])
            loss_0 = loss_0_x_ligand + loss_0_x_pocket + loss_0_h

        nll = loss_t + loss_0 + kl_prior

        # print("loss", nll)
        nll = nll.mean()

        # 反向传播
        nll.backward()

        # 更新参数
        optimizer.step()

        # 累加损失
        total_loss += nll.item()
        pbar.set_postfix(nll_loss=nll.item())  # 更新进度条的后缀信息

    print(f'Epoch {epoch}, Average NLL Loss: {total_loss / len(train_loader)}')

    # 验证阶段
    cddpm.eval()  # 设置模型为评估模式
    val_loss = 0
    with torch.no_grad():  # 不需要计算梯度
        for batch in val_loader:
            # batch = {key: batch[key].to(device) for key in batch}
            
            ligand, pocket = get_ligand_and_pocket(batch, virtual_nodes)
            delta_log_px, error_t_lig, error_t_pocket, SNR_weight, \
            loss_0_x_ligand, loss_0_x_pocket, loss_0_h, neg_log_const_0, \
            kl_prior, log_pN, t_int, xh_lig_hat, info = cddpm(ligand, pocket, return_info=True)

            if loss_type == 'l2':
                actual_ligand_size = ligand['size'] - ligand['num_virtual_atoms'] if virtual_nodes else ligand['size']

                # normalize loss_t
                denom_lig = x_dims * actual_ligand_size + \
                            cddpm.atom_nf * ligand['size']
                error_t_lig = error_t_lig / denom_lig
                denom_pocket = (x_dims + cddpm.residue_nf) * pocket['size']
                error_t_pocket = error_t_pocket / denom_pocket
                loss_t = 0.5 * (error_t_lig + error_t_pocket)

                # normalize loss_0
                loss_0_x_ligand = loss_0_x_ligand / (x_dims * actual_ligand_size)
                loss_0_x_pocket = loss_0_x_pocket / (x_dims * pocket['size'])
                loss_0 = loss_0_x_ligand + loss_0_x_pocket + loss_0_h

            nll = loss_t + loss_0 + kl_prior
            nll = nll.mean()  # 将 nll 转换为标量
            val_loss += nll.item()

    print(f'Epoch {epoch}, Validation Loss: {val_loss / len(val_loader)}')

    # 每个 epoch 后保存模型
    torch.save(cddpm.state_dict(), os.path.join(save_dir, f'cddpm_epoch_{epoch}.pth'))

# 最终测试阶段
cddpm.eval()  # 设置模型为评估模式
test_loss = 0
with torch.no_grad():  # 不需要计算梯度
    for batch in test_loader:
        # batch = {key: batch[key].to(device) for key in batch}

        ligand, pocket = get_ligand_and_pocket(batch, virtual_nodes)
        delta_log_px, error_t_lig, error_t_pocket, SNR_weight, \
        loss_0_x_ligand, loss_0_x_pocket, loss_0_h, neg_log_const_0, \
        kl_prior, log_pN, t_int, xh_lig_hat, info = cddpm(ligand, pocket, return_info=True)

        if loss_type == 'l2':
            actual_ligand_size = ligand['size'] - ligand['num_virtual_atoms'] if virtual_nodes else ligand['size']

            # normalize loss_t
            denom_lig = x_dims * actual_ligand_size + \
                        cddpm.atom_nf * ligand['size']
            error_t_lig = error_t_lig / denom_lig
            denom_pocket = (x_dims + cddpm.residue_nf) * pocket['size']
            error_t_pocket = error_t_pocket / denom_pocket
            loss_t = 0.5 * (error_t_lig + error_t_pocket)

            # normalize loss_0
            loss_0_x_ligand = loss_0_x_ligand / (x_dims * actual_ligand_size)
            loss_0_x_pocket = loss_0_x_pocket / (x_dims * pocket['size'])
            loss_0 = loss_0_x_ligand + loss_0_x_pocket + loss_0_h

        nll = loss_t + loss_0 + kl_prior
        nll = nll.mean()  # 将 nll 转换为标量
        test_loss += nll.item()

print(f'Test Loss: {test_loss / len(test_loader)}')

# 最终保存模型
torch.save(cddpm.state_dict(), os.path.join(save_dir, 'cddpm_final.pth'))

Epoch 0:   0%|          | 0/58 [00:00<?, ?it/s]

Epoch 0, Average NLL Loss: 0.7029596896007143


Epoch 0, Validation Loss: 1.2497879894156205
Test Loss: 1.2392394244670868
